<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети (однослойная)
####Вариант 5

In [ ]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [ ]:
#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 2):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 2):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.5):
  return 1 if S >= T else 0

#=================================================================
def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

#=================================================================
#функции для формирования таблицы данных
def x1(v):
  return math.sin(math.log(abs(v[0])) / v[2])

def x2(v):
  return math.cos(v[1]**3)

def x3(v):
  return math.sin(math.tan(v[0] / v[1]) - 3**(v[2]))

def y1(x):
  return (x[0] / x[1]) * math.sqrt(abs(x[0]))

def y2(x):
  return math.sqrt(abs(math.sin(x[2])))

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -10, end = 10):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, elem = 100):
  vector = []
  for i in range(elem):
    vector.append(function(generationRandomVector()))
  return vector

#подставление x,v в выходные функции
def addVariants(function, x, elem = 100):
  vector = []
  for i in range(elem):
    vector.append(function(x[i]))
  return vector

In [ ]:
#Коэффициент обучения (точность) [0;1]
learningRate = 0.1
#Количество эпох
layer = 10
#количество нейронов на каждом слое
#neurons = [2,3,1]
neurons = 2
#количество входов/выходов соответственно
countX = 3
countY = 2

networkCharact = [layer, neurons, learningRate]
inpOutCounts = [countX, countY]
inputFunctions = [x1, x2, x3]
outputFunctions = [y1, y2]

#=================================================================
data = []
for i in range(len(inputFunctions)):
  data.append(createVariants(inputFunctions[i]))
for i in range(len(outputFunctions)):
  data.append(addVariants(outputFunctions[i], np.array(data[0:3]).T))

data_df = pd.DataFrame(np.array(data).T, columns=['x1', 'x2', 'x3', 'y1', 'y2'])

In [ ]:
data_df

In [ ]:
#charactNet = [кол-во слоёв, кол-во нейронов, коэф. обучения]
#sizeSystem = [кол-во входов, кол-во выходов]

def deltaRule(function, data, charactNet, sizeSystem):
  countLayers = charactNet[0]
  countNeurons = charactNet[1]
  nabla = charactNet[2]

  indexOrder = [i for i in range(0, len(data))] #список всех индексов строк таблицы данных

  #si = [0] * sizeSystem[1]
  yi = [0] * sizeSystem[1]
  epsi = [0] * sizeSystem[1]
  level_i = 0
  k = 0

  #временная инициализация весов (только для однослойной сети!)
  wi = [[randomNumber() for i in range(sizeSystem[0])] for j in range(countNeurons)] #синаптические веса

  while(level_i < countLayers): #цикл по ЭПОХАМ!!!!!!!

    print("------------------------------------------------------------------------------------------------------------------")
    print("\tЭпоха №" + str(level_i + 1))
    random.shuffle(indexOrder)

    while(k < len(data)): #цикл по строкам таблицы с данными

      print('================================================')
      print("Матрица весов для " + str(k + 1) + "-й итерации:")
      print(pd.DataFrame(wi))
      print("\nДля X = " + str(np.array(data.iloc[indexOrder[k], 0:sizeSystem[0]]).T) + '\n')

      eps = 0
      si = [0] * sizeSystem[1]

      for i in range(countNeurons): #по нейронам
        for j in range(sizeSystem[0]): #по входам
          si[i] += data[data.columns[j]][indexOrder[k]] * wi[i][j]

        yi[i] = function(si[i])
        epsi[i] = data.iloc[indexOrder[k], sizeSystem[0] + i] - yi[i]
        eps += (epsi[i]**2) / countNeurons

      rez_i = pd.DataFrame([si, yi, data.iloc[k, sizeSystem[0]:], epsi], index=['S', 'Y расч.', 'Y теор.', 'EPS'],
                           columns=['neuron ' + str(i+1) for i in range(sizeSystem[1])])

      print(rez_i)
      print("\nСКО: " + str(eps) + "\n\n")

      #вычисление весов для следующей итерации:
      for i in range(countNeurons):
        for j in range(sizeSystem[0]):
          wi[i][j] = wi[i][j] + nabla * epsi[i] * data[data.columns[j]][indexOrder[k]]

      k += 1

    level_i += 1
    k = 0

  #

In [ ]:
#table = pd.DataFrame(it.product([0,1], repeat=countX), columns=['X' + str(i + 1) for i in range(countX)])
#table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
#table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

#deltaRule(hyperbolicTangentFunction, table, networkCharact, inpOutCounts)
#deltaRule(sigmoidFunction, table, layer, neurons, learningRate)
#deltaRule(thresholdFunction, table, layer, neurons, learningRate)

deltaRule(hyperbolicTangentFunction, data_df, networkCharact, inpOutCounts)

Выходные данные были обрезаны до нескольких последних строк (5000).
          0         1         2
0 -0.955674  0.111755 -0.220511
1  0.125946 -0.076855  0.097536

Для X = [0.08431748 0.73006776 0.76731268]

         neuron 1  neuron 2
S       -0.168192  0.029350
Y расч. -0.083899  0.014674
Y теор.  4.817625  0.714440
EPS      0.117435  0.818515

СКО: 0.3418785475423657


Матрица весов для 8-й итерации:
          0         1         2
0 -0.954683  0.120328 -0.211500
1  0.132847 -0.017098  0.160341

Для X = [-0.11004786  0.91595685  0.93492101]

         neuron 1  neuron 2
S        0.017540  0.119626
Y расч.  0.008770  0.059742
Y теор.  0.549155  0.899666
EPS     -0.048626  0.837227

СКО: 0.3516564389918682


Матрица весов для 9-й итерации:
          0         1         2
0 -0.954148  0.115874 -0.216046
1  0.123634  0.059588  0.238615

Для X = [ 0.14630015 -0.67787881 -0.87701434]

         neuron 1  neuron 2
S       -0.028665 -0.231575
Y расч. -0.014332 -0.115273
Y теор. -2.872168  0.